In [206]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [207]:
import re

import geopandas as gpd
import requests,json


from shapely import geos
from shapely.geometry import Point
import fiona
from fiona.crs import from_epsg,from_string

from geopandas import GeoDataFrame
from shapely.geometry import Polygon
from geopandas import GeoSeries


from shapely import geometry
from shapely.geometry import shape,LineString,Point,Polygon

import osmnx as ox
import networkx as nx

import folium

from folium import IFrame

In [208]:
def rest(address):
    try:
        url = 'https://api.tomtom.com/search/2/geocode/{}.json?&key=h2n93GznJsnsDHtv2vPpdT9GmJijp9o9&countrySet=TWN&language=zh-TW&limit=1'.format(address)
        response = requests.get(url)
        data = response.text
        js = json.loads(str(data))
        return js['results'][0]['position']
    except Exception as e:
        print(f'An Error occurred: {e}')
        return {'lat':0,'lon':0}
    
def trans(gdf):
    gdf.crs = {'init' :'epsg:3824'} # 避免資料沒設，這邊再重新給一次
    gdf=gdf.to_crs({'init' :'epsg:4326'})
    return gdf

def utf2asc(s):
    return str(str(s).encode('ascii', 'xmlcharrefreplace'))[2:-1]

heading3 = """<h3>{}</h3>""".format

In [209]:
df_get_back=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\get back.xlsx')
print(df_get_back.shape)
df_get_back.head()

df_get_back=df_get_back.loc[:,['id','lon','lat','check']]

(133014, 6)


In [210]:
pop=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\台灣202012年村里人口.xlsx')
print(pop.shape)

(7760, 5)


In [211]:
site=['1G30林口中山店']
date=['202101']
start=['190726']

import warnings
warnings.filterwarnings("ignore")

import os
os.chdir('C:/Users/RT009223/TestriteGroup/滲透率專案/01_raw data/TLW社區店/'+site[0]+'/'+date[0])

In [212]:
df01=pd.read_csv(r'ff交易紀錄.csv',header=None,names=['id','site','date','AMT','trans_cnt','tick_cnt','card_type'])
print(df01.shape)
print(df01.head())

df01['temp']=0
df01.loc[df01.card_type=='05',['temp']]=1

df01_2=df01.groupby(['site','temp'])['AMT'].sum().reset_index(name='AMT')
df01_2['share']=df01_2.AMT/df01_2.AMT.sum()

df_final01=df01_2.copy()
print(df_final01)

(3131, 7)
              id  site                     date    AMT  trans_cnt  tick_cnt  \
0  2190449830011  1G30  2021-01-01 00:00:00.000   75.0          1         1   
1  2020883080102  1G30  2021-01-01 00:00:00.000  138.0          1         1   
2  2230512050011  1G30  2021-01-01 00:00:00.000  285.0          1         1   
3  2043144550015  1G30  2021-01-02 00:00:00.000  190.0          1         1   
4  2070785990016  1G30  2021-01-02 00:00:00.000  245.0          1         1   

  card_type  
0        00  
1        00  
2        00  
3        00  
4        00  
   site  temp         AMT     share
0  1G30     0  4048661.53  0.934637
1  1G30     1   283140.42  0.065363


In [213]:
df02=pd.read_csv(r'一般卡會員活躍程度.csv',header=None,names=['id','type'])
print(df02.shape)
print(df02.head())

df02.type=df02.type.apply(lambda x:str(x))
df02.type=df02.type.replace('1','type1:new')
df02.type=df02.type.replace('2','type2:active')
df02.type=df02.type.replace('3','type3:1yr~3yr')
df02.type=df02.type.replace('4','type4:3yr~')

df02_2=df02.groupby(['type'])['id'].count().reset_index(name='n')
df02_2['share']=df02_2.n/df02_2.n.sum()

df_final02=df02_2.copy()
print(df_final02)

(2332, 2)
               id  type
0  19960102000619     2
1  19960129000220     2
2  19960131000679     2
3  19960131000808     4
4  19960203000137     2
            type     n     share
0      type1:new   126  0.054031
1   type2:active  1599  0.685678
2  type3:1yr~3yr   174  0.074614
3     type4:3yr~   433  0.185678


In [214]:
df03=pd.read_csv(r'一般卡會員會員樣貌.csv',header=None,names=['id','name','gender','birth','status','age','age_type'])
print(df03.shape)
print(df03.head())


df03=pd.merge(df03,df02,on='id')

df03_2=df03.groupby(['age_type'])['id'].count().reset_index(name='n_all')
df03_2['share_all']=df03_2.n_all/df03_2.n_all.sum()
df03_3=df03[df03.type=='type1:new'].groupby(['age_type'])['id'].count().reset_index(name='n_new')
df03_3['share_new']=df03_3.n_new/df03_3.n_new.sum()

df03_4=pd.merge(df03_2,df03_3,on='age_type',how='outer')
df03_4=df03_4.fillna(0)
print(df03_4.share_all.sum())
print(df03_4.share_new.sum())

df_final03=df03_4.copy()
print(df_final03)

(2332, 7)
               id name gender                    birth  status   age  age_type
0  19991031000106  賴國隆      M  1963-03-15 00:00:00.000       1  57.0  10:55-59
1  19991118002145  黃志彬      M  1971-09-30 00:00:00.000       1  49.0  08:45-49
2  20030318032833  楊惟如      F  1975-10-29 00:00:00.000       2  45.0  08:45-49
3  19991127004045  陳美涼      F  1961-01-15 00:00:00.000       2  60.0  11:60-64
4  20011220000639  馮婉宜      F  1980-04-24 00:00:00.000       1  40.0  07:40-44
0.9999999999999999
1.0
    age_type  n_all  share_all  n_new  share_new
0   02:15-19      6   0.002573      2   0.015873
1   03:20-24     28   0.012007     11   0.087302
2   04:25-29     83   0.035592     23   0.182540
3   05:30-34    159   0.068182     15   0.119048
4   06:35-39    308   0.132075     18   0.142857
5   07:40-44    363   0.155660     11   0.087302
6   08:45-49    353   0.151372      7   0.055556
7   09:50-54    268   0.114923      5   0.039683
8   10:55-59    269   0.115352      7   0.055556
9  

In [215]:
df03_5=df03.groupby(['gender'])['id'].count().reset_index(name='n_all')
df03_5['share_all']=df03_5.n_all/df03_5.n_all.sum()
df03_6=df03[df03.type=='type1:new'].groupby(['gender'])['id'].count().reset_index(name='n_new')
df03_6['share_new']=df03_6.n_new/df03_6.n_new.sum()

df03_7=pd.merge(df03_5,df03_6,on='gender',how='outer')
df03_7=df03_7.fillna(0)
print(df03_7.share_all.sum())
print(df03_7.share_new.sum())

df_final04=df03_7.copy()
df_final04

1.0
1.0


,gender,n_all,share_all,n_new,share_new
0,F,941,0.403516,60,0.47619
1,M,1391,0.596484,66,0.52381


In [216]:
df04=pd.read_csv(r'sales一般卡交易紀錄.csv',header=None,names=['id','card','site','date','sku','class1','class1_name','AMT'])
print(df04.shape)
df04.head()

(10649, 8)


,id,card,site,date,sku,class1,class1_name,AMT
0,20010922002788,2014329080122,1G30,2021-01-19 00:00:00.000,16004868,66,寢具,437.0
1,20191004000241,2992144939019,1G30,2021-01-08 00:00:00.000,14284609,66,寢具,1905.0
2,20191024000790,2992151176018,1G30,2021-01-05 00:00:00.000,16042077,66,寢具,4277.0
3,20040206000534,2035891330019,1G30,2021-01-18 00:00:00.000,14291085,66,寢具,666.0
4,20200102000696,2752496820010,1G30,2021-01-02 00:00:00.000,16017510,66,寢具,569.0


In [217]:
df04_2=pd.merge(df04,df03,on='id',how='left')
print(df04_2.shape)
df04_2.head()

(10649, 15)


,id,card,site,date,sku,class1,class1_name,AMT,name,gender,birth,status,age,age_type,type
0,20010922002788,2014329080122,1G30,2021-01-19 00:00:00.000,16004868,66,寢具,437.0,陳泰和,M,1964-02-29 00:00:00.000,2,56.0,10:55-59,type2:active
1,20191004000241,2992144939019,1G30,2021-01-08 00:00:00.000,14284609,66,寢具,1905.0,徐義芳,M,1979-06-06 00:00:00.000,1,41.0,07:40-44,type2:active
2,20191024000790,2992151176018,1G30,2021-01-05 00:00:00.000,16042077,66,寢具,4277.0,盧貴珠,F,1911-02-01 00:00:00.000,1,109.0,99:其他,type2:active
3,20040206000534,2035891330019,1G30,2021-01-18 00:00:00.000,14291085,66,寢具,666.0,尤基政,M,1979-11-06 00:00:00.000,2,41.0,07:40-44,type4:3yr~
4,20200102000696,2752496820010,1G30,2021-01-02 00:00:00.000,16017510,66,寢具,569.0,梅豫芬,F,1964-05-29 00:00:00.000,1,56.0,10:55-59,type4:3yr~


In [218]:
df04_3=df04_2.groupby(['class1_name'])['id'].nunique().reset_index(name='id_all')
df04_3['id_all_rate']=df04_3.id_all/df03.shape[0]
print(df04_3.shape)

df04_4=df04_2[df04_2.type=='type1:new'].groupby(['class1_name'])['id'].nunique().reset_index(name='id_new')
df04_4['id_new_rate']=df04_4.id_new/df03[df03.type=='type1:new'].shape[0]
print(df04_4.shape)
df04_4.head()

(26, 3)
(26, 3)


,class1_name,id_new,id_new_rate
0,五金,19,0.150794
1,休閒樂活&園藝,12,0.095238
2,傢俱,7,0.055556
3,地材,7,0.055556
4,家用品,38,0.301587


In [219]:
dept_name=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\01_raw data\TLW社區店\大分類中文名稱.xlsx')
df04_5=pd.merge(df04_3,df04_4,on='class1_name',how='outer')

df_final05=pd.merge(dept_name,df04_5,how='left')
df_final05=df_final05.fillna(0)
print(df_final05.shape)
df_final05.head()

(92, 5)


,class1_name,id_all,id_all_rate,id_new,id_new_rate
0,油漆/防水,289.0,0.123928,11.0,0.087302
1,膠類,199.0,0.085334,9.0,0.071429
2,窗飾/壁飾,56.0,0.024014,5.0,0.039683
3,地材,104.0,0.044597,7.0,0.055556
4,燈源/燈具,438.0,0.187822,25.0,0.198413


In [220]:
df_01=pd.read_csv('一般卡會員(有完整地址)_'+start[0]+'.csv',error_bad_lines=False,header=None)
df_01.columns=['id','addr','lon','lat']
df_01.shape

(12273, 4)

In [221]:
pattern = re.compile(r'(..[市,縣])(.{1,2}[市,區,鄉,鎮])(.{2,3}[里,村])')
location=df_01.addr.apply(lambda x:pattern.findall(x))

addr01=[]
addr02=[]
addr03=[]
for i in location:
    if len(i)==1:
        addr01.append(i[0][0])
        addr02.append(i[0][1])
        addr03.append(i[0][2])        
    else:
        addr01.append(np.nan)
        addr02.append(np.nan)
        addr03.append(np.nan)  

In [222]:
df_01['縣市']=addr01
df_01['行政區']=addr02
df_01['村里']=addr03

In [223]:
df_02=df_01.dropna()
df_02.shape

(12270, 7)

In [224]:
gdf=gpd.read_file(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\geo data\VILLAGE_MOI_1090423.shp',encoding='utf-8')
gdf=gdf.loc[:,['COUNTYNAME','TOWNNAME','VILLNAME','geometry']]
gdf=trans(gdf)
gdf=gdf.dropna().reset_index(drop=True)
print(gdf.shape)

gdf.columns=['縣市','行政區','村里','geometry']
gdf['縣市']=list(gdf['縣市'].apply(lambda x:x.replace('臺','台')))
gdf['行政區']=list(gdf['行政區'].apply(lambda x:x.replace('臺','台')))
gdf['村里']=list(gdf['村里'].apply(lambda x:x.replace('臺','台')))

df_02['縣市']=list(df_02['縣市'].apply(lambda x:x.replace('臺','台')))
df_02['行政區']=list(df_02['行政區'].apply(lambda x:x.replace('臺','台')))
df_02['村里']=list(df_02['村里'].apply(lambda x:x.replace('臺','台')))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('舘','館'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('舘','館'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('双','雙'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('双','雙'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('欍','瓊'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('溝垻里','溝埧里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('崙峰里','崙峯里'))
gdf.head()

(7760, 4)


,縣市,行政區,村里,geometry
0,高雄市,林園區,中門里,"POLYGON ((120.36772 22.49564, 120.36427 22.498..."
1,高雄市,林園區,港埔里,"POLYGON ((120.37325 22.49123, 120.37147 22.492..."
2,高雄市,旗津區,上竹里,"POLYGON ((120.28976 22.57317, 120.28577 22.577..."
3,高雄市,旗津區,中興里,"POLYGON ((120.30320 22.55568, 120.30284 22.557..."
4,連江縣,南竿鄉,津沙村,"POLYGON ((119.91684 26.14109, 119.91683 26.141..."


In [225]:
site1=df_02[-(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].groupby(['村里'])['行政區'].nunique().reset_index(name='n').sort_values(by='n',ascending=False).reset_index(drop=True)
site1.head(5)

,村里,n
0,公?里,2
1,?林里,2
2,?寮里,1
3,磚?里,1
4,瓦?里,1


In [226]:
for i in site1[site1.n>0]['村里']:
    print(i, df_02[df_02['村里']==i]['行政區'].unique())

公?里 ['板橋區' '西區']
?林里 ['大園區' '竹東鎮']
?寮里 ['樹林區']
磚?里 ['彰化市']
瓦?里 ['中和區']
永?里 ['三峽區']
新?里 ['永和區']
南?里 ['彰化市']
公西里 ['龜山區' '林口區']
五?里 ['新店區']
?廷里 ['土城區']
下?里 ['彰化市']
上?里 ['竹東鎮']
?行里 ['永康區']
?福村 ['民雄鄉']
?洲里 ['永康區']
糖?里 ['萬華區']


In [227]:
site2=site1[site1.n>0]['村里'][0]
site3=df_02[df_02['村里']==site2]['行政區'].unique()

for i in site3:
    print(i,list(gdf[gdf['行政區']==i]['村里'][gdf[gdf['行政區']==i]['村里'].str.contains('公西里')]))

板橋區 []
西區 []


In [228]:
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('糖部里','糖廍里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('糖?里','糖廍里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('磚?里','磚磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('磚瑤里','磚磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('下?里','下廍里'))

df_02.loc[df_02['行政區']=='彰化市',['村里']]=df_02.loc[df_02['行政區']=='彰化市',['村里']].apply(lambda x:x.replace('南?里','南瑶里'))
df_02.loc[df_02['行政區']=='官田區',['村里']]=df_02.loc[df_02['行政區']=='官田區',['村里']].apply(lambda x:x.replace('南?里','南廍里'))
#df_02['村里']=df_02['村里'].apply(lambda x:x.replace('南?里','南瑶里'))


df_02['村里']=df_02['村里'].apply(lambda x:x.replace('崙?里','崙峯里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?子里','廍子里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('石?村','石硦村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('上?里','上館里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('永?里','永館里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('瓦?村','瓦磘村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('瓦?里','瓦磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('五?里','五峯里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?行里','塩行里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?廷里','峯廷里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?興里','塩興里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?廷里','峯廷里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('公?里','公館里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?洲里','塩洲里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?龍里','雙龍里'))

#df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?林里','雞林里'))
df_02.loc[df_02['行政區']=='竹東鎮',['村里']]=df_02.loc[df_02['行政區']=='竹東鎮',['村里']].apply(lambda x:x.replace('?林里','雞林里'))

df_02.loc[df_02['行政區']=='大園區',['村里']]=df_02.loc[df_02['行政區']=='大園區',['村里']].apply(lambda x:x.replace('?林里','菓林里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?寮里','獇寮里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('新?里','新廍里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('蔗?里','蔗廍里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?埔里','坔埔里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?榔里','槺榔里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?江里','晋江里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?福村','雙福村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('爪?里','爪峯里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('舊?村','舊館村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('南?村','南館村'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('灰?里','灰磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('三?村','三廍村'))


df_02.loc[df_02['行政區']=='將軍區',['村里']]=df_02.loc[df_02['行政區']=='將軍區',['村里']].apply(lambda x:x.replace('忠興里','忠嘉里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('下部里','下廍里'))
df_02.loc[df_02['行政區']=='埔鹽鄉',['村里']]=df_02.loc[df_02['行政區']=='埔鹽鄉',['村里']].apply(lambda x:x.replace('?子村','廍子村'))
df_02.loc[df_02['行政區']=='四湖鄉',['村里']]=df_02.loc[df_02['行政區']=='四湖鄉',['村里']].apply(lambda x:x.replace('?子村','萡子村'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?田里','塩田里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('山?里','山脚里'))

df_02.loc[df_02['行政區']=='北埔鄉',['村里']]=df_02.loc[df_02['行政區']=='北埔鄉',['村里']].apply(lambda x:x.replace('水?村','水祭村'))
df_02.loc[df_02['行政區']=='北埔鄉',['村里']]=df_02.loc[df_02['行政區']=='北埔鄉',['村里']].apply(lambda x:x.replace('水？村','水祭村'))
df_02.loc[df_02['行政區']=='瑞芳區',['村里']]=df_02.loc[df_02['行政區']=='瑞芳區',['村里']].apply(lambda x:x.replace('?洞里','濂洞里'))

df_02.loc[df_02['行政區']=='埔心鄉',['村里']]=df_02.loc[df_02['行政區']=='埔心鄉',['村里']].apply(lambda x:x.replace('埤?村','埤脚村'))

In [229]:
#合
df_02_1=df_02[(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)

#不合
df_02_2=df_02[-(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
print(df_02_1.shape)
print(df_02_2.shape)

(12269, 7)
(1, 7)


In [230]:
df_02_2.lon=df_02_2.lon.apply(lambda x:float(x))
df_02_2.lat=df_02_2.lat.apply(lambda x:float(x))

In [231]:
if df_02_2.shape[0]>0:
    geom=[Point(data.lon,data.lat) for idx, data in df_02_2.iterrows()]
    crs = {'init': 'epsg:4326'}
    df_02_2 = gpd.GeoDataFrame(df_02_2, crs=crs, geometry=geom)
    print(df_02_2.shape)
    df_02_3 = gpd.tools.sjoin(df_02_2,gdf, how="inner", op='intersects')
    print(df_02_3.shape)
    df_02_3=df_02_3.iloc[:,[0,1,2,3,9,10,11]].reset_index(drop=True)
    df_02_3.columns=['id','addr','lon','lat','縣市','行政區','村里']
    df_02_3.head()
else:
    df_02_3=df_02_2.copy()

(1, 8)
(1, 12)


In [232]:
df_02=pd.concat([df_02_1,df_02_3]).reset_index(drop=True)
df_02.shape

(12270, 7)

In [233]:
df_03=df_01[-df_01.id.isin(df_02.id)].dropna(subset=['lon'])
df_03.shape

(3, 7)

In [234]:
pattern = re.compile(r'(..[市,縣])(.{2,3}[市,區,鄉,鎮])(.{1,2}[里,村])')
location=df_03.addr.apply(lambda x:pattern.findall(x))

addr01=[]
addr02=[]
addr03=[]
for i in location:
    if len(i)==1:
        addr01.append(i[0][0])
        addr02.append(i[0][1])
        addr03.append(i[0][2])        
    else:
        addr01.append(np.nan)
        addr02.append(np.nan)
        addr03.append(np.nan)  

In [235]:
df_03['縣市']=addr01
df_03['行政區']=addr02
df_03['村里']=addr03

In [236]:
#合
df_03_1=df_03[(df_03['縣市']+df_03['行政區']+df_03['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
print(df_03_1.shape)

#不合
df_03_2=df_03[-(df_03['縣市']+df_03['行政區']+df_03['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
print(df_03_2.shape)


(1, 7)
(2, 7)


In [237]:
def to_float(x):
    try:
        return float(x)
    except:
        return 0

In [238]:
df_03_2.lon=df_03_2.lon.apply(to_float)
df_03_2.lat=df_03_2.lat.apply(to_float)

if df_03_2.shape[0]>0:
    geom=[Point(data.lon,data.lat) for idx, data in df_03_2.iterrows()]
    crs = {'init': 'epsg:4326'}
    df_03_2 = gpd.GeoDataFrame(df_03_2, crs=crs, geometry=geom)
    print(df_03_2.shape)
    df_03_3 = gpd.tools.sjoin(df_03_2,gdf, how="inner", op='intersects')
    print(df_03_3.shape)
    df_03_3=df_03_3.iloc[:,[0,1,2,3,9,10,11]].reset_index(drop=True)
    df_03_3.columns=['id','addr','lon','lat','縣市','行政區','村里']
    print(df_03_3.shape)
else:
    df_03_3=df_03_2.copy()

(2, 8)
(2, 12)
(2, 7)


In [239]:
df_03=pd.concat([df_03_1,df_03_3]).reset_index(drop=True)

In [240]:
df_01=pd.concat([df_02,df_03]).reset_index(drop=True)
df_01.shape

(12273, 7)

In [241]:
df_04=pd.read_excel(r'一般卡會員(無完整地址)_'+start[0]+'.xlsx')
df_04.shape

(2074, 3)

In [242]:
df_04['addr']=df_04.addr.apply(lambda x:str(x))

In [243]:
df_04_1=df_04[df_04.addr.apply(lambda x:len(x)>9)].reset_index(drop=True)
df_04_1.shape

(526, 3)

In [244]:
pattern = re.compile(r'..[市,縣].+[路,街].+')
location=df_04_1.addr.apply(lambda x:pattern.findall((x)))

addr01=[]
for i in location:
    if len(i)==1:
        addr01.append(i[0])      
    else:
        addr01.append(np.nan)

In [245]:
df_04_1.addr=addr01
df_04_1=df_04_1.dropna(subset=['addr']).reset_index(drop=True)
df_04_1.shape

(345, 3)

In [246]:
df_04_1['lon']=float(0)
df_04_1['lat']=float(0)
df_04_1['check']=0
df_04_1.head()

,id,addr,zone,lon,lat,check
0,20101031000215,台北市中山區清江路21巷18號2樓,104.0,0.0,0.0,0
1,19981228000379,新北市三重區文化路1-1號,241.0,0.0,0.0,0
2,20171010000274,新北市泰山區復興北路1號14樓-4,243.0,0.0,0.0,0
3,19991229000211,新北市林口區松山路155巷1號3樓,244.0,0.0,0.0,0
4,20010223000214,新北市林口區湖子路４８之６號,244.0,0.0,0.0,0


In [247]:
df_04_1=pd.merge(df_04_1.loc[:,['id','addr','zone']],df_get_back,on='id',how='left')

In [248]:
df_04_1.lon[df_04_1.lon.isna()]=0
df_04_1.lat[df_04_1.lat.isna()]=0
df_04_1.check[df_04_1.check.isna()]=0

In [249]:
df_04_1.head()

,id,addr,zone,lon,lat,check
0,20101031000215,台北市中山區清江路21巷18號2樓,104.0,0.0000,0.00000,0.0
1,19981228000379,新北市三重區文化路1-1號,241.0,0.0000,0.00000,0.0
2,20171010000274,新北市泰山區復興北路1號14樓-4,243.0,0.0000,0.00000,0.0
3,19991229000211,新北市林口區松山路155巷1號3樓,244.0,121.3867,25.07897,1.0
4,20010223000214,新北市林口區湖子路４８之６號,244.0,0.0000,0.00000,0.0


In [250]:
df_zone=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\郵遞區代碼.xlsx')

In [251]:
df_04_1=pd.merge(df_04_1,df_zone,on='zone',how='left')

In [252]:
geom=[Point(data.lon,data.lat) for idx, data in df_04_1.iterrows()]
crs = {'init': 'epsg:4326'}
df_04_1 = gpd.GeoDataFrame(df_04_1, crs=crs, geometry=geom)
df_04_1.shape

(345, 9)

In [253]:
df_04_1= gpd.tools.sjoin(df_04_1,gdf, how="left", op='intersects')
df_04_1.shape

(345, 13)

In [254]:
df_04_1.head()

,id,addr,zone,lon,lat,check,縣市_left,行政區_left,geometry,index_right,縣市_right,行政區_right,村里
0,20101031000215,台北市中山區清江路21巷18號2樓,104.0,0.0000,0.00000,0.0,台北市,中山區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN
1,19981228000379,新北市三重區文化路1-1號,241.0,0.0000,0.00000,0.0,新北市,三重區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN
2,20171010000274,新北市泰山區復興北路1號14樓-4,243.0,0.0000,0.00000,0.0,新北市,泰山區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN
3,19991229000211,新北市林口區松山路155巷1號3樓,244.0,121.3867,25.07897,1.0,新北市,林口區,POINT (121.38670 25.07897),6962.0,新北市,林口區,西林里
4,20010223000214,新北市林口區湖子路４８之６號,244.0,0.0000,0.00000,0.0,新北市,林口區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN


In [255]:
# 無 map 無 zone
df_04_1_1=df_04_1[(df_04_1['行政區_left'].isna()&df_04_1['村里'].isna())]
print(df_04_1_1.shape)

# 無 map 有 zone
df_04_1_2=df_04_1[(-(df_04_1['行政區_left'].isna())&(df_04_1['村里'].isna()))]
print(df_04_1_2.shape)

# 有 map 無有 zone
df_04_1_3=df_04_1[(df_04_1['行政區_left'].isna()&(-df_04_1['村里'].isna()))]
print(df_04_1_3.shape)

# 有 map 有 zone
df_04_1_4=df_04_1[(-(df_04_1['行政區_left'].isna())&(-df_04_1['村里'].isna()))]
print(df_04_1_4.shape)

(2, 13)
(260, 13)
(2, 13)
(81, 13)


In [256]:
df_04_1_4_1=df_04_1_4[(df_04_1_4['行政區_left']==df_04_1_4['行政區_right'])]
print(df_04_1_4_1.shape)

df_04_1_4_2=df_04_1_4[-(df_04_1_4['行政區_left']==df_04_1_4['行政區_right'])]
print(df_04_1_4_2.shape)

(71, 13)
(10, 13)


In [257]:
df_04_1_1=df_04_1_1.loc[:,['id']]

df_04_1_2=df_04_1_2.loc[:,['id','縣市_left','行政區_left']]
df_04_1_2.columns=['id','縣市','行政區']

df_04_1_3=df_04_1_3.loc[:,['id','縣市_right','行政區_right','村里']]
df_04_1_3.columns=['id','縣市','行政區','村里']

df_04_1_4_1=df_04_1_4_1.loc[:,['id','縣市_right','行政區_right','村里']]
df_04_1_4_1.columns=['id','縣市','行政區','村里']

df_04_1_4_2=df_04_1_4_2.loc[:,['id','縣市_left','行政區_left']]
df_04_1_4_2.columns=['id','縣市','行政區']

In [258]:
df_04_2=pd.concat([df_04_1_4_1,df_04_1_3,df_04_1_4_2,df_04_1_2,df_04_1_1])
df_04_2.shape

(345, 4)

In [259]:
df_04_2.drop_duplicates(subset=['id']).shape

(345, 4)

In [260]:
df_04_3=df_04[-df_04.id.isin(df_04_2.id)]
df_04_3.shape

(1729, 3)

In [261]:
df_04_3=pd.merge(df_04_3,df_zone,on='zone',how='left')

In [262]:
df_04_3_1=df_04_3[-df_04_3['行政區'].isna()]
print(df_04_3_1.shape)

df_04_3_2=df_04_3[df_04_3['行政區'].isna()]
print(df_04_3_2.shape)

(351, 5)
(1378, 5)


In [263]:
df_04_3_1=df_04_3_1.loc[:,['id','縣市','行政區']]
df_04_3_2=df_04_3_2.loc[:,['id']]

In [264]:
df_04=pd.concat([df_04_2,df_04_3_1,df_04_3_2])
print(df_04.shape)
print(df_04.drop_duplicates(subset=['id']).shape)

(2074, 4)
(2074, 4)


In [265]:
result=pd.concat([df_01,df_04]).reset_index(drop=True)
result.loc[result['縣市'].isna(),['縣市']]='NA'
result.loc[result['行政區'].isna(),['行政區']]='NA'
result.loc[result['村里'].isna(),['村里']]='NA'
print(result.shape)
result.head()

(14347, 7)


,id,addr,lon,lat,縣市,行政區,村里
0,20000401005271,臺北市文山區樟樹里6鄰忠順街一段26巷1弄3號 ...,121.559444,24.983885,台北市,文山區,樟樹里
1,20000311000186,桃園市龜山區大湖里10鄰文化二路34巷14弄12號 ...,121.364784,25.059128,桃園市,龜山區,大湖里
2,20161027001075,桃園市龜山區文化里3鄰文昌一街27號 ...,121.368465,25.058161,桃園市,龜山區,文化里
3,19960229000343,新北市林口區南勢里33鄰文化北路一段526巷51號 ...,121.368472,25.078462,新北市,林口區,南勢里
4,19961213000020,新北市林口區湖南里7鄰文化三路二段306號 ...,121.377826,25.084242,新北市,林口區,湖南里


In [266]:
have=result.shape[0]
lost=(result['村里']=='NA').sum()
have_share=1-lost/have
print(site[0],'have_share',have_share)

1G30林口中山店 have_share 0.8605283334495016


In [267]:
df_sp01=pd.merge(df02,result)
df_sp02=df_sp01.groupby(['縣市','行政區','村里'])['id'].count().reset_index()


df_sp02['縣市'][df_sp02['村里']=='NA']='NA'
df_sp02['行政區'][df_sp02['村里']=='NA']='NA'
df_sp02['縣市'][(df_sp02['縣市']!='新北市')&(df_sp02['村里']!='NA')]='其他'
df_sp02['行政區'][(df_sp02['縣市']!='新北市')&(df_sp02['村里']!='NA')]='其他'
df_sp02['村里'][(df_sp02['行政區']=='其他')&(df_sp02['村里']!='NA')]='其他'
df_sp02=df_sp02.groupby(['縣市','行政區','村里'])['id'].sum().reset_index()

df_sp02=df_sp02.sort_values(by='id',ascending=False).reset_index(drop=True)
df_sp02=pd.concat([df_sp02.loc[(-df_sp02['村里'].isin(['其他','NA'])),:],df_sp02.loc[(df_sp02['村里']=='其他'),:],df_sp02.loc[(df_sp02['村里']=='NA'),:]]).reset_index(drop=True)
df_sp02['rate']=df_sp02['id']/df_sp02['id'].sum()
df_sp02.id.sum()

2332

In [268]:
df_sp01=pd.merge(df02,result)
df_sp02=df_sp01.groupby(['縣市','行政區','村里','type'])['id'].count().reset_index()

loss_rate_01=df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type1:new')].sum()/df_sp02['id'][(df_sp02['type']=='type1:new')].sum()
loss_rate_02=df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type2:active')].sum()/df_sp02['id'][(df_sp02['type']=='type2:active')].sum()
loss_rate_03=df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type3:1yr~3yr')].sum()/df_sp02['id'][(df_sp02['type']=='type3:1yr~3yr')].sum()
loss_rate_04=df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type4:3yr~')].sum()/df_sp02['id'][(df_sp02['type']=='type4:3yr~')].sum()

df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type1:new')]=df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type1:new')]/loss_rate_01
df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type2:active')]=df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type2:active')]/loss_rate_02
df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type3:1yr~3yr')]=df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type3:1yr~3yr')]/loss_rate_03
df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type4:3yr~')]=df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type4:3yr~')]/loss_rate_04

df_sp02=df_sp02[df_sp02['村里']!='NA'].reset_index(drop=True)
df_sp02['縣市'][df_sp02['縣市']!='新北市']='其他'
df_sp02['行政區'][df_sp02['縣市']!='新北市']='其他'
df_sp02['村里'][df_sp02['縣市']!='新北市']='其他'
df_sp02=df_sp02.groupby(['縣市','行政區','村里'])['id'].sum().reset_index()
df_sp02=df_sp02.sort_values(by='id',ascending=False).reset_index(drop=True)
df_sp02=pd.concat([df_sp02.loc[(df_sp02['村里']!='其他'),:],df_sp02.loc[(df_sp02['村里']=='其他'),:]]).reset_index(drop=True)
df_sp02['rate']=df_sp02['id']/df_sp02['id'].sum()
df_sp02.id.sum()

2332.0

In [269]:
df_sp01=pd.merge(df02,result)
df_sp02=df_sp01.groupby(['縣市','行政區','村里','type'])['id'].count().reset_index()

loss_rate_01=df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type1:new')].sum()/df_sp02['id'][(df_sp02['type']=='type1:new')].sum()
loss_rate_02=df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type2:active')].sum()/df_sp02['id'][(df_sp02['type']=='type2:active')].sum()
loss_rate_03=df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type3:1yr~3yr')].sum()/df_sp02['id'][(df_sp02['type']=='type3:1yr~3yr')].sum()
loss_rate_04=df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type4:3yr~')].sum()/df_sp02['id'][(df_sp02['type']=='type4:3yr~')].sum()

df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type1:new')]=df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type1:new')]/loss_rate_01
df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type2:active')]=df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type2:active')]/loss_rate_02
df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type3:1yr~3yr')]=df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type3:1yr~3yr')]/loss_rate_03
df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type4:3yr~')]=df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type4:3yr~')]/loss_rate_04

df_sp02=df_sp02[df_sp02['行政區']!='NA'].reset_index(drop=True)
df_sp02['縣市'][df_sp02['縣市']!='新北市']='其他'
df_sp02['行政區'][df_sp02['縣市']!='新北市']='其他'
df_sp02['村里'][df_sp02['縣市']!='新北市']='其他'
df_sp02=df_sp02.groupby(['縣市','行政區','type'])['id'].sum().reset_index()
def replace_place(x):
    religion=['林口區','龜山區']
    if x =='NA':
        x=x
        return x
    elif x not in religion:
        x='其他'
        return x
    else:
        return x

df_sp02['行政區']=df_sp02['行政區'].apply(replace_place)
df_sp02=df_sp02.groupby(['行政區','type'])['id'].sum().reset_index()
df_sp02=pd.concat([df_sp02.loc[(df_sp02['行政區']!='其他'),:],df_sp02.loc[(df_sp02['行政區']=='其他'),:]]).reset_index(drop=True)
df_sp02['rate']=df_sp02['id']/df_sp02['id'].sum()
df_sp02.id.sum()

2331.999999999998

In [270]:
'''
df_sp01=pd.merge(df02,result)
df_sp01['村里2']=df_sp01['村里']
def replace_place02(x):
    religion=list(gdf.loc[(gdf['縣市']=='新北市'),'村里'].values)
    if x =='NA':
        x=x
        return x
    elif x not in religion:
        x='其他'
        return x
    else:
        return x

df_sp01['村里2']=df_sp01['村里2'].apply(replace_place02)
df_sp02=df_sp01.groupby(['縣市','行政區','村里2','type'])['id'].count().reset_index()
df_sp02['行政區'][df_sp02['村里2']=='其他']='其他'
df_sp02['縣市'][df_sp02['村里2']=='其他']='其他'
df_sp02['行政區'][df_sp02['村里2']=='NA']='NA'
df_sp02['縣市'][df_sp02['村里2']=='NA']='NA'
df_sp02=df_sp02.groupby(['縣市','行政區','村里2','type'])['id'].sum().reset_index()
print(df_sp02.id.sum())

df_sp02_loss=df_sp02['id'][df_sp02['村里2']!='NA'].sum()/df_sp02['id'].sum()
df_sp02['id']=df_sp02['id']/df_sp02_loss
df_sp02=df_sp02[df_sp02['村里2']!='NA'].reset_index(drop=True)
df_sp02
'''

"\ndf_sp01=pd.merge(df02,result)\ndf_sp01['村里2']=df_sp01['村里']\ndef replace_place02(x):\n    religion=list(gdf.loc[(gdf['縣市']=='新北市'),'村里'].values)\n    if x =='NA':\n        x=x\n        return x\n    elif x not in religion:\n        x='其他'\n        return x\n    else:\n        return x\n\ndf_sp01['村里2']=df_sp01['村里2'].apply(replace_place02)\ndf_sp02=df_sp01.groupby(['縣市','行政區','村里2','type'])['id'].count().reset_index()\ndf_sp02['行政區'][df_sp02['村里2']=='其他']='其他'\ndf_sp02['縣市'][df_sp02['村里2']=='其他']='其他'\ndf_sp02['行政區'][df_sp02['村里2']=='NA']='NA'\ndf_sp02['縣市'][df_sp02['村里2']=='NA']='NA'\ndf_sp02=df_sp02.groupby(['縣市','行政區','村里2','type'])['id'].sum().reset_index()\nprint(df_sp02.id.sum())\n\ndf_sp02_loss=df_sp02['id'][df_sp02['村里2']!='NA'].sum()/df_sp02['id'].sum()\ndf_sp02['id']=df_sp02['id']/df_sp02_loss\ndf_sp02=df_sp02[df_sp02['村里2']!='NA'].reset_index(drop=True)\ndf_sp02\n"

In [271]:
#df_sp02.groupby(['縣市','行政區','村里2'])['id'].sum().reset_index()

In [272]:
'''
df_sp01=pd.merge(df02,result)
df_sp01['行政區2']=df_sp01['行政區']
def replace_place(x):
    religion=['林口區','龜山區']
    if x =='NA':
        x=x
        return x
    elif x not in religion:
        x='其他'
        return x
    else:
        return x

df_sp01['行政區2']=df_sp01['行政區2'].apply(replace_place)
df_sp02=df_sp01.groupby(['type','行政區2'])['id'].count().reset_index()
df_sp02
'''

"\ndf_sp01=pd.merge(df02,result)\ndf_sp01['行政區2']=df_sp01['行政區']\ndef replace_place(x):\n    religion=['林口區','龜山區']\n    if x =='NA':\n        x=x\n        return x\n    elif x not in religion:\n        x='其他'\n        return x\n    else:\n        return x\n\ndf_sp01['行政區2']=df_sp01['行政區2'].apply(replace_place)\ndf_sp02=df_sp01.groupby(['type','行政區2'])['id'].count().reset_index()\ndf_sp02\n"

In [273]:
pop['縣市']=pop['縣市'].str.replace('\xa0','')
pop['行政區']=pop['行政區'].str.replace('\xa0','')
pop['村里']=pop['村里'].str.replace('\xa0','')

In [274]:
gdf['縣市']=list(gdf['縣市'].apply(lambda x:x.replace('臺','台')))
gdf['行政區']=list(gdf['行政區'].apply(lambda x:x.replace('臺','台')))
gdf['村里']=list(gdf['村里'].apply(lambda x:x.replace('臺','台')))
pop['縣市']=list(pop['縣市'].apply(lambda x:x.replace('臺','台')))
pop['行政區']=list(pop['行政區'].apply(lambda x:x.replace('臺','台')))
pop['村里']=list(pop['村里'].apply(lambda x:x.replace('臺','台')))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('舘','館'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('舘','館'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('双','雙'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('双','雙'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('欍','瓊'))

pop['村里']=pop['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

pop['村里']=pop['村里'].apply(lambda x:x.replace('欍','瓊'))


pop['村里']=pop['村里'].apply(lambda x:x.replace('溝垻里','溝埧里'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('崙峰里','崙峯里'))

In [275]:
result[-(result['行政區']+result['村里']).isin(gdf['行政區']+gdf['村里'])]['村里'].unique()

array(['NA'], dtype=object)

In [276]:
result['村里']=result['村里'].apply(lambda x:x.replace('舘','館'))
result['村里']=result['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
result['村里']=result['村里'].apply(lambda x:x.replace('双','雙'))
result['村里']=result['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
result['村里']=result['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
result['村里']=result['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))
result['村里']=result['村里'].apply(lambda x:x.replace('欍','瓊'))

In [277]:
result2=result.groupby(['縣市','行政區','村里'])['id'].count().reset_index()
result2.columns=['縣市','行政區','村里','會員數']
result2.shape

(2490, 4)

In [278]:
result.shape[0]

14347

In [279]:
result2=pd.merge(pop,result2,how='outer')

In [280]:
result2=pd.merge(pop,result2,how='outer')
result2=result2[-(-result2['村里'].isin(pop['村里'])&-(result2['村里']=='NA'))].reset_index(drop=True)
result2.shape

(7856, 6)

In [281]:
result2[-(result2['行政區']+result2['村里']).isin(gdf['行政區']+gdf['村里'])]['村里'].unique()

array(['NA'], dtype=object)

In [282]:
result2['會員數'].sum()

14347.0

In [283]:
gdf_final=pd.merge(gdf,result2,how='right')

In [284]:
gdf_final['會員數'][gdf_final['會員數'].isna()]=0

In [285]:
gdf_final['site']=gdf_final['行政區']+'\n'+gdf_final['村里']
gdf_final['ratio']=gdf_final['會員數']/gdf_final['戶口']
print(gdf_final.shape)
gdf_final.head()

(7856, 9)


,縣市,行政區,村里,geometry,戶口,人口,會員數,site,ratio
0,高雄市,林園區,中門里,"POLYGON ((120.36772 22.49564, 120.36427 22.498...",1130.0,2871.0,0.0,林園區\n中門里,0.0
1,高雄市,林園區,港埔里,"POLYGON ((120.37325 22.49123, 120.37147 22.492...",823.0,2040.0,0.0,林園區\n港埔里,0.0
2,高雄市,旗津區,上竹里,"POLYGON ((120.28976 22.57317, 120.28577 22.577...",1372.0,3202.0,0.0,旗津區\n上竹里,0.0
3,高雄市,旗津區,中興里,"POLYGON ((120.30320 22.55568, 120.30284 22.557...",1065.0,2858.0,0.0,旗津區\n中興里,0.0
4,連江縣,南竿鄉,津沙村,"POLYGON ((119.91684 26.14109, 119.91683 26.141...",114.0,459.0,0.0,南竿鄉\n津沙村,0.0


In [286]:
north1=['新北市','台北市','桃園市']

In [287]:
area=[north1]

In [288]:
print(gdf_final.shape)

location=area[0]
gdf_final_sub=gdf_final.loc[gdf_final['縣市'].isin(location),['縣市','行政區','村里','會員數','戶口','ratio']]
print(gdf_final_sub.shape)

gdf_final_sub_out=gdf_final.loc[-gdf_final['縣市'].isin(location),['縣市','行政區','村里','會員數','戶口','ratio']]
print(gdf_final_sub_out.shape)

(7856, 9)
(2038, 6)
(5818, 6)


In [289]:
NA_value=gdf_final_sub_out[gdf_final_sub_out['行政區']=='NA']['會員數'].sum()
NA_value

not_NA_value=gdf_final_sub_out[gdf_final_sub_out['行政區']!='NA']['會員數'].sum()
not_NA_value

1185.0

In [290]:
NA_value

1380.0

In [291]:
df_final06=gdf_final_sub.groupby(['縣市','行政區'])['會員數','戶口'].sum().reset_index()
df_final06['ratio']=df_final06['會員數']/df_final06['戶口']

NA_value=gdf_final_sub_out[gdf_final_sub_out['行政區']=='NA']['會員數'].sum()

not_NA_value=gdf_final_sub_out[gdf_final_sub_out['行政區']!='NA']['會員數'].sum()

df_final06=pd.concat([df_final06,pd.DataFrame({'縣市':['其他地區','其他地區NA'],'會員數':[not_NA_value,NA_value]})])
print(df_final06.shape)
df_final06.head()

(56, 5)


,縣市,行政區,會員數,戶口,ratio
0,台北市,中山區,249.0,100790.0,0.002470
1,台北市,中正區,112.0,65795.0,0.001702
2,台北市,信義區,149.0,88952.0,0.001675
3,台北市,內湖區,255.0,109891.0,0.002320
4,台北市,北投區,163.0,98131.0,0.001661


In [292]:
df_final06['會員數'].sum()==gdf_final['會員數'].sum()

True

In [293]:
df_final07=gdf_final_sub.copy()

NA_value=gdf_final_sub_out[gdf_final_sub_out['村里']=='NA']['會員數'].sum()

not_NA_value=gdf_final_sub_out[gdf_final_sub_out['村里']!='NA']['會員數'].sum()

df_final07=pd.concat([df_final07,pd.DataFrame({'縣市':['其他地區','其他地區NA'],'會員數':[not_NA_value,NA_value]})])
print(df_final07.shape)
df_final07.head()

(2040, 6)


,縣市,行政區,村里,會員數,戶口,ratio
42,桃園市,大溪區,仁義里,0.0,1381.0,0.000000
43,桃園市,八德區,瑞德里,1.0,1565.0,0.000639
870,桃園市,龜山區,楓樹里,17.0,1693.0,0.010041
871,桃園市,桃園區,大興里,4.0,2600.0,0.001538
872,桃園市,桃園區,忠義里,5.0,1654.0,0.003023


In [294]:
df_final01['店點']=site[0]
df_final01=df_final01.set_index('店點').reset_index()

df_final02['店點']=site[0]
df_final02=df_final02.set_index('店點').reset_index()

df_final03['店點']=site[0]
df_final03=df_final03.set_index('店點').reset_index()

df_final04['店點']=site[0]
df_final04=df_final04.set_index('店點').reset_index()

df_final05['店點']=site[0]
df_final05=df_final05.set_index('店點').reset_index()

df_final06['店點']=site[0]
df_final06=df_final06.set_index('店點').reset_index()

df_final07['店點']=site[0]
df_final07=df_final07.set_index('店點').reset_index()

In [295]:
df_final05_2=df_final05.copy()
df_final05_2['']=''
df_final05_2=df_final05_2.set_index('').reset_index()


df_final06_2=df_final06.copy()
df_final06_2['']=''
df_final06_2=df_final06_2.set_index('').reset_index()

df_final07_2=df_final07.copy()
df_final07_2['']=''
df_final07_2=df_final07_2.set_index('').reset_index()

In [296]:
df_final05a=df_final05.copy()
df_final06a=df_final06.copy()
df_final07a=df_final07.copy()
df_final05b=df_final05_2.copy()
df_final06b=df_final06_2.copy()
df_final07b=df_final07_2.copy()

In [297]:
df_final06

,店點,縣市,行政區,會員數,戶口,ratio
0,1G30林口中山店,台北市,中山區,249.0,100790.0,0.002470
1,1G30林口中山店,台北市,中正區,112.0,65795.0,0.001702
2,1G30林口中山店,台北市,信義區,149.0,88952.0,0.001675
3,1G30林口中山店,台北市,內湖區,255.0,109891.0,0.002320
4,1G30林口中山店,台北市,北投區,163.0,98131.0,0.001661
5,1G30林口中山店,台北市,南港區,68.0,48010.0,0.001416
6,1G30林口中山店,台北市,士林區,256.0,108148.0,0.002367
7,1G30林口中山店,台北市,大同區,95.0,52132.0,0.001822
8,1G30林口中山店,台北市,大安區,189.0,121152.0,0.001560
9,1G30林口中山店,台北市,文山區,119.0,107669.0,0.001105


In [298]:
site=['1G30林口中山店','1G32桃園大業店','1G33竹北文興店','1G34蘆洲集賢店','1G35板橋新埔店','1G36新營金華店','1G37永康復國店','1G38大安安和店','1G39頭份中央店','1G40三民澄清店','1G45基隆義一店','1G44嘉義興業西店','1G42永和得和店','1G46台中復興店','1G41岡山大仁店','1G43三重集美店','1G47龍潭北龍店']
start=['190726','190903','191123','191204','200328','200321','200411','200530','200711','200912','201107','201128','201205','201219','201231','210116','210123']

north1=['新北市','台北市','桃園市']
north2=['新竹縣','新竹市','桃園市','新北市']
north3=['新竹縣','新竹市','桃園市']
north4=['新竹縣','新竹市','苗栗縣']
north5=['基隆市']
west=['南投縣','彰化縣','台中市']
south1=['台南市','嘉義縣','嘉義市']
south2=['高雄市','屏東縣']
south3=['台南市','高雄市']
east=[]
area=[north1,north2,north3,north1,north1,south1,south1,north1,north4,south2,north5,south1,north1,west,south3,north1,north2]

In [299]:
print(int(date[0][2:])*100+40)
end_site=(np.array(list(map(int,start)))<int(date[0][2:])*100+40).sum()
end_site

210140


17

In [300]:
for i in range(1,end_site):
    print(site[i])

1G32桃園大業店
1G33竹北文興店
1G34蘆洲集賢店
1G35板橋新埔店
1G36新營金華店
1G37永康復國店
1G38大安安和店
1G39頭份中央店
1G40三民澄清店
1G45基隆義一店
1G44嘉義興業西店
1G42永和得和店
1G46台中復興店
1G41岡山大仁店
1G43三重集美店
1G47龍潭北龍店


# 看不懂的迴圈起始點

In [301]:
for a in range(1,end_site):
    os.chdir('C:/Users/RT009223/TestriteGroup/滲透率專案/01_raw data/TLW社區店/'+site[a]+'/'+date[0])
    print(site[a],date[0])
    df01=pd.read_csv(r'ff交易紀錄.csv',header=None,names=['id','site','date','AMT','trans_cnt','tick_cnt','card_type'])
    print(df01.shape)
    df01.head()
    df01['temp']=0
    df01.loc[df01.card_type=='05',['temp']]=1
    df01_2=df01.groupby(['site','temp'])['AMT'].sum().reset_index(name='AMT')
    df01_2['share']=df01_2.AMT/df01_2.AMT.sum()
    df_final01_another=df01_2.copy()


    df02=pd.read_csv(r'一般卡會員活躍程度.csv',header=None,names=['id','type'])
    print('當月一般卡會員',df02.shape)
    df02.head()
    df02.type=df02.type.apply(lambda x:str(x))
    df02.type=df02.type.replace('1','type1:new')
    df02.type=df02.type.replace('2','type2:active')
    df02.type=df02.type.replace('3','type3:1yr~3yr')
    df02.type=df02.type.replace('4','type4:3yr~')
    df02_2=df02.groupby(['type'])['id'].count().reset_index(name='n')
    df02_2['share']=df02_2.n/df02_2.n.sum()
    df_final02_another=df02_2.copy()

    df03=pd.read_csv(r'一般卡會員會員樣貌.csv',header=None,names=['id','name','gender','birth','status','age','age_type'])
    print(df03.shape)
    df03.head()
    df03=pd.merge(df03,df02,on='id')
    df03_2=df03.groupby(['age_type'])['id'].count().reset_index(name='n_all')
    df03_2['share_all']=df03_2.n_all/df03_2.n_all.sum()
    df03_3=df03[df03.type=='type1:new'].groupby(['age_type'])['id'].count().reset_index(name='n_new')
    df03_3['share_new']=df03_3.n_new/df03_3.n_new.sum()
    df03_4=pd.merge(df03_2,df03_3,on='age_type',how='outer')
    df03_4=df03_4.fillna(0)
    print(df03_4.share_all.sum())
    print(df03_4.share_new.sum())
    df_final03_another=df03_4.copy()

    df03_5=df03.groupby(['gender'])['id'].count().reset_index(name='n_all')
    df03_5['share_all']=df03_5.n_all/df03_5.n_all.sum()
    df03_6=df03[df03.type=='type1:new'].groupby(['gender'])['id'].count().reset_index(name='n_new')
    df03_6['share_new']=df03_6.n_new/df03_6.n_new.sum()
    df03_7=pd.merge(df03_5,df03_6,on='gender',how='outer')
    df03_7=df03_7.fillna(0)
    print(df03_7.share_all.sum())
    print(df03_7.share_new.sum())
    df_final04_another=df03_7.copy()

    df04=pd.read_csv(r'sales一般卡交易紀錄.csv',header=None,names=['id','card','site','date','sku','class1','class1_name','AMT'])
    print(df04.shape)
    df04.head()
    df04_2=pd.merge(df04,df03,on='id',how='left')
    print(df04_2.shape)
    df04_2.head()
    df04_3=df04_2.groupby(['class1_name'])['id'].nunique().reset_index(name='id_all')
    df04_3['id_all_rate']=df04_3.id_all/df03.shape[0]
    print(df04_3.shape)
    df04_3.head()
    df04_4=df04_2[df04_2.type=='type1:new'].groupby(['class1_name'])['id'].nunique().reset_index(name='id_new')
    df04_4['id_new_rate']=df04_4.id_new/df03[df03.type=='type1:new'].shape[0]
    print(df04_4.shape)
    df04_4.head()
    dept_name=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\01_raw data\TLW社區店\大分類中文名稱.xlsx')
    df04_5=pd.merge(df04_3,df04_4,on='class1_name',how='outer')
    df04_6=pd.merge(dept_name,df04_5,how='left')
    df04_6=df04_6.fillna(0)
    df_final05_another=df04_6.copy()



    df_01=pd.read_csv('一般卡會員(有完整地址)_'+start[a]+'.csv',error_bad_lines=False,header=None)
    df_01.columns=['id','addr','lon','lat']
    print('一般卡會員(有完整地址)',df_01.shape)
    pattern = re.compile(r'(..[市,縣])(.{1,2}[市,區,鄉,鎮])(.{2,3}[里,村])')
    location=df_01.addr.apply(lambda x:pattern.findall(x))

    addr01=[]
    addr02=[]
    addr03=[]
    for i in location:
        if len(i)==1:
            addr01.append(i[0][0])
            addr02.append(i[0][1])
            addr03.append(i[0][2])        
        else:
            addr01.append(np.nan)
            addr02.append(np.nan)
            addr03.append(np.nan)  
    df_01['縣市']=addr01
    df_01['行政區']=addr02
    df_01['村里']=addr03
    df_02=df_01.dropna()
    print(df_02.shape)

    print(gdf.shape)

    gdf.columns=['縣市','行政區','村里','geometry']
    gdf['縣市']=list(gdf['縣市'].apply(lambda x:x.replace('臺','台')))
    gdf['行政區']=list(gdf['行政區'].apply(lambda x:x.replace('臺','台')))
    gdf['村里']=list(gdf['村里'].apply(lambda x:x.replace('臺','台')))

    df_02['縣市']=list(df_02['縣市'].apply(lambda x:x.replace('臺','台')))
    df_02['行政區']=list(df_02['行政區'].apply(lambda x:x.replace('臺','台')))
    df_02['村里']=list(df_02['村里'].apply(lambda x:x.replace('臺','台')))

    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('舘','館'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('舘','館'))
    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('双','雙'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('双','雙'))
    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('欍','瓊'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('溝垻里','溝埧里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('崙峰里','崙峯里'))
    gdf.head()
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('糖部里','糖廍里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('糖?里','糖廍里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('磚?里','磚磘里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('磚瑤里','磚磘里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('下?里','下廍里'))

    df_02.loc[df_02['行政區']=='彰化市',['村里']]=df_02.loc[df_02['行政區']=='彰化市',['村里']].apply(lambda x:x.replace('南?里','南瑶里'))
    df_02.loc[df_02['行政區']=='官田區',['村里']]=df_02.loc[df_02['行政區']=='官田區',['村里']].apply(lambda x:x.replace('南?里','南廍里'))



    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('崙?里','崙峯里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?子里','廍子里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('石?村','石硦村'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('上?里','上館里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('永?里','永館里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('瓦?村','瓦磘村'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('瓦?里','瓦磘里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('五?里','五峯里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?行里','塩行里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?廷里','峯廷里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?興里','塩興里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?廷里','峯廷里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('公?里','公館里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?洲里','塩洲里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?龍里','雙龍里'))

    #df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?林里','雞林里'))
    df_02.loc[df_02['行政區']=='竹東鎮',['村里']]=df_02.loc[df_02['行政區']=='竹東鎮',['村里']].apply(lambda x:x.replace('?林里','雞林里'))

    df_02.loc[df_02['行政區']=='大園區',['村里']]=df_02.loc[df_02['行政區']=='大園區',['村里']].apply(lambda x:x.replace('?林里','菓林里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?寮里','獇寮里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('新?里','新廍里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('蔗?里','蔗廍里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?埔里','坔埔里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?榔里','槺榔里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?江里','晋江里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?福村','雙福村'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('爪?里','爪峯里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('舊?村','舊館村'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('南?村','南館村'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('灰?里','灰磘里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('三?村','三廍村'))


    df_02.loc[df_02['行政區']=='將軍區',['村里']]=df_02.loc[df_02['行政區']=='將軍區',['村里']].apply(lambda x:x.replace('忠興里','忠嘉里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('下部里','下廍里'))
    df_02.loc[df_02['行政區']=='埔鹽鄉',['村里']]=df_02.loc[df_02['行政區']=='埔鹽鄉',['村里']].apply(lambda x:x.replace('?子村','廍子村'))
    df_02.loc[df_02['行政區']=='四湖鄉',['村里']]=df_02.loc[df_02['行政區']=='四湖鄉',['村里']].apply(lambda x:x.replace('?子村','萡子村'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?田里','塩田里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('山?里','山脚里'))

    df_02.loc[df_02['行政區']=='北埔鄉',['村里']]=df_02.loc[df_02['行政區']=='北埔鄉',['村里']].apply(lambda x:x.replace('水?村','水祭村'))
    df_02.loc[df_02['行政區']=='北埔鄉',['村里']]=df_02.loc[df_02['行政區']=='北埔鄉',['村里']].apply(lambda x:x.replace('水？村','水祭村'))
    df_02.loc[df_02['行政區']=='瑞芳區',['村里']]=df_02.loc[df_02['行政區']=='瑞芳區',['村里']].apply(lambda x:x.replace('?洞里','濂洞里'))

    df_02.loc[df_02['行政區']=='埔心鄉',['村里']]=df_02.loc[df_02['行政區']=='埔心鄉',['村里']].apply(lambda x:x.replace('埤?村','埤脚村'))
    
    
    #合
    df_02_1=df_02[(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)

    #不合
    df_02_2=df_02[-(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
    print(df_02_1.shape)
    print(df_02_2.shape)
    df_02_2.lon=df_02_2.lon.apply(lambda x:float(x))
    df_02_2.lat=df_02_2.lat.apply(lambda x:float(x))
    if df_02_2.shape[0]>0:        
        geom=[Point(data.lon,data.lat) for idx, data in df_02_2.iterrows()]
        crs = {'init': 'epsg:4326'}
        df_02_2 = gpd.GeoDataFrame(df_02_2, crs=crs, geometry=geom)
        print(df_02_2.shape)
        df_02_3 = gpd.tools.sjoin(df_02_2,gdf, how="inner", op='intersects')
        print(df_02_3.shape)
        df_02_3=df_02_3.iloc[:,[0,1,2,3,9,10,11]].reset_index(drop=True)
        df_02_3.columns=['id','addr','lon','lat','縣市','行政區','村里']
        df_02_3.head()
    else:
        df_02_3=df_02_2.copy()
    df_02=pd.concat([df_02_1,df_02_3]).reset_index(drop=True)
    print(df_02.shape)
    df_03=df_01[-df_01.id.isin(df_02.id)].dropna(subset=['lon'])
    print(df_03.shape)
    pattern = re.compile(r'(..[市,縣])(.{2,3}[市,區,鄉,鎮])(.{1,2}[里,村])')
    location=df_03.addr.apply(lambda x:pattern.findall(x))

    addr01=[]
    addr02=[]
    addr03=[]
    for i in location:
        if len(i)==1:
            addr01.append(i[0][0])
            addr02.append(i[0][1])
            addr03.append(i[0][2])        
        else:
            addr01.append(np.nan)
            addr02.append(np.nan)
            addr03.append(np.nan)  
    df_03['縣市']=addr01
    df_03['行政區']=addr02
    df_03['村里']=addr03
    #合
    df_03_1=df_03[(df_03['縣市']+df_03['行政區']+df_03['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
    print(df_03_1.shape)

    #不合
    df_03_2=df_03[-(df_03['縣市']+df_03['行政區']+df_03['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
    print(df_03_2.shape)
    def to_float(x):
        try:
            return float(x)
        except:
            return 0
    df_03_2.lon=df_03_2.lon.apply(to_float)
    df_03_2.lat=df_03_2.lat.apply(to_float)
    if df_03_2.shape[0]>0:
        geom=[Point(data.lon,data.lat) for idx, data in df_03_2.iterrows()]
        crs = {'init': 'epsg:4326'}
        df_03_2 = gpd.GeoDataFrame(df_03_2, crs=crs, geometry=geom)
        print(df_03_2.shape)
        df_03_3 = gpd.tools.sjoin(df_03_2,gdf, how="inner", op='intersects')
        print(df_03_3.shape)
        df_03_3=df_03_3.iloc[:,[0,1,2,3,9,10,11]].reset_index(drop=True)
        df_03_3.columns=['id','addr','lon','lat','縣市','行政區','村里']
        print(df_03_3.shape)
    else:
        df_03_3=df_03_2.copy()
    df_03=pd.concat([df_03_1,df_03_3]).reset_index(drop=True)
    df_01=pd.concat([df_02,df_03]).reset_index(drop=True)
    print('一般卡會員(有完整地址)_new',df_01.shape)
    
    
    df_04=pd.read_excel('一般卡會員(無完整地址)_'+start[a]+'.xlsx')
    print('一般卡會員(無完整地址)',df_04.shape)
    df_04['addr']=df_04.addr.apply(lambda x:str(x))
    df_04_1=df_04[df_04.addr.apply(lambda x:len(x)>9)].reset_index(drop=True)
    print(df_04_1.shape)
    pattern = re.compile(r'..[市,縣].+[路,街].+')
    location=df_04_1.addr.apply(lambda x:pattern.findall((x)))

    addr01=[]
    for i in location:
        if len(i)==1:
            addr01.append(i[0])      
        else:
            addr01.append(np.nan)
    df_04_1.addr=addr01
    df_04_1=df_04_1.dropna(subset=['addr']).reset_index(drop=True)
    print(df_04_1.shape)
    df_04_1['lon']=float(0)
    df_04_1['lat']=float(0)
    df_04_1['check']=0
    df_04_1.head()


    print(df_get_back.shape)
    df_get_back.head()
    df_04_1=pd.merge(df_04_1.loc[:,['id','addr','zone']],df_get_back,on='id',how='left')
    df_04_1.lon[df_04_1.lon.isna()]=0
    df_04_1.lat[df_04_1.lat.isna()]=0
    df_04_1.check[df_04_1.check.isna()]=0
    df_04_1=pd.merge(df_04_1,df_zone,on='zone',how='left')
    geom=[Point(data.lon,data.lat) for idx, data in df_04_1.iterrows()]
    crs = {'init': 'epsg:4326'}
    df_04_1 = gpd.GeoDataFrame(df_04_1, crs=crs, geometry=geom)
    df_04_1.shape
    df_04_1= gpd.tools.sjoin(df_04_1,gdf, how="left", op='intersects')
    print(df_04_1.shape)

    # 無 map 無 zone
    df_04_1_1=df_04_1[(df_04_1['行政區_left'].isna()&df_04_1['村里'].isna())]
    print(df_04_1_1.shape)

    # 無 map 有 zone
    df_04_1_2=df_04_1[(-(df_04_1['行政區_left'].isna())&(df_04_1['村里'].isna()))]
    print(df_04_1_2.shape)

    # 有 map 無有 zone
    df_04_1_3=df_04_1[(df_04_1['行政區_left'].isna()&(-df_04_1['村里'].isna()))]
    print(df_04_1_3.shape)

    # 有 map 有 zone
    df_04_1_4=df_04_1[(-(df_04_1['行政區_left'].isna())&(-df_04_1['村里'].isna()))]
    print(df_04_1_4.shape)

    df_04_1_4_1=df_04_1_4[(df_04_1_4['行政區_left']==df_04_1_4['行政區_right'])]
    print(df_04_1_4_1.shape)

    df_04_1_4_2=df_04_1_4[-(df_04_1_4['行政區_left']==df_04_1_4['行政區_right'])]
    print(df_04_1_4_2.shape)

    df_04_1_1=df_04_1_1.loc[:,['id']]

    df_04_1_2=df_04_1_2.loc[:,['id','縣市_left','行政區_left']]
    df_04_1_2.columns=['id','縣市','行政區']

    df_04_1_3=df_04_1_3.loc[:,['id','縣市_right','行政區_right','村里']]
    df_04_1_3.columns=['id','縣市','行政區','村里']

    df_04_1_4_1=df_04_1_4_1.loc[:,['id','縣市_right','行政區_right','村里']]
    df_04_1_4_1.columns=['id','縣市','行政區','村里']

    df_04_1_4_2=df_04_1_4_2.loc[:,['id','縣市_left','行政區_left']]
    df_04_1_4_2.columns=['id','縣市','行政區']

    df_04_2=pd.concat([df_04_1_4_1,df_04_1_3,df_04_1_4_2,df_04_1_2,df_04_1_1])
    print(df_04_2.shape)
    print(df_04_2.drop_duplicates(subset=['id']).shape)
    df_04_3=df_04[-df_04.id.isin(df_04_2.id)]
    print(df_04_3.shape)

    df_04_3=pd.merge(df_04_3,df_zone,on='zone',how='left')
    df_04_3_1=df_04_3[-df_04_3['行政區'].isna()]
    print(df_04_3_1.shape)

    df_04_3_2=df_04_3[df_04_3['行政區'].isna()]
    print(df_04_3_2.shape)
    df_04_3_1=df_04_3_1.loc[:,['id','縣市','行政區']]
    df_04_3_2=df_04_3_2.loc[:,['id']]
    df_04=pd.concat([df_04_2,df_04_3_1,df_04_3_2])
    print('一般卡會員(無完整地址)_new',df_04.shape)
    print(df_04.drop_duplicates(subset=['id']).shape)
    result=pd.concat([df_01,df_04]).reset_index(drop=True)
    result.loc[result['縣市'].isna(),['縣市']]='NA'
    result.loc[result['行政區'].isna(),['行政區']]='NA'
    result.loc[result['村里'].isna(),['村里']]='NA'
    print('全部一般卡會員_new',result.shape)
    result.head()
    have=result.shape[0]
    lost=(result['村里']=='NA').sum()
    have_share=1-lost/have
    print(site[a],'have_share',have_share)
    
    ########################
    if site[a]=='1G45基隆義一店':
        area02='基隆市'
        writer02 = pd.ExcelWriter('C:/Users/RT009223/TestriteGroup/滲透率專案/06_TLW社區店/month/'+date[0][2:]+'/'+site[a]+'分析('+date[0]+')_Iris用.xlsx', engine='xlsxwriter')
        df_sp01=pd.merge(df02,result)
        df_sp02=df_sp01.groupby(['縣市','行政區','村里'])['id'].count().reset_index()

        df_sp02['縣市'][df_sp02['村里']=='NA']='NA'
        df_sp02['行政區'][df_sp02['村里']=='NA']='NA'
        df_sp02['縣市'][(df_sp02['縣市']!=area02)&(df_sp02['村里']!='NA')]='其他'
        df_sp02['行政區'][(df_sp02['縣市']!=area02)&(df_sp02['村里']!='NA')]='其他'
        df_sp02['村里'][(df_sp02['行政區']=='其他')&(df_sp02['村里']!='NA')]='其他'
        df_sp02=df_sp02.groupby(['縣市','行政區','村里'])['id'].sum().reset_index()

        df_sp02=df_sp02.sort_values(by='id',ascending=False).reset_index(drop=True)
        df_sp02=pd.concat([df_sp02.loc[(-df_sp02['村里'].isin(['其他','NA'])),:],df_sp02.loc[(df_sp02['村里']=='其他'),:],df_sp02.loc[(df_sp02['村里']=='NA'),:]]).reset_index(drop=True)
        df_sp02['rate']=df_sp02['id']/df_sp02['id'].sum()
        df_sp02.to_excel(writer02, sheet_name='顧客來源村里占比(獨立NA)',index=0)   

        df_sp01=pd.merge(df02,result)
        df_sp02=df_sp01.groupby(['縣市','行政區','村里','type'])['id'].count().reset_index()

        loss_rate_01=df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type1:new')].sum()/df_sp02['id'][(df_sp02['type']=='type1:new')].sum()
        loss_rate_02=df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type2:active')].sum()/df_sp02['id'][(df_sp02['type']=='type2:active')].sum()
        loss_rate_03=df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type3:1yr~3yr')].sum()/df_sp02['id'][(df_sp02['type']=='type3:1yr~3yr')].sum()
        loss_rate_04=df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type4:3yr~')].sum()/df_sp02['id'][(df_sp02['type']=='type4:3yr~')].sum()

        df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type1:new')]=df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type1:new')]/loss_rate_01
        df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type2:active')]=df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type2:active')]/loss_rate_02
        df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type3:1yr~3yr')]=df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type3:1yr~3yr')]/loss_rate_03
        df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type4:3yr~')]=df_sp02['id'][(df_sp02['村里']!='NA')&(df_sp02['type']=='type4:3yr~')]/loss_rate_04

        df_sp02=df_sp02[df_sp02['村里']!='NA'].reset_index(drop=True)
        df_sp02['縣市'][df_sp02['縣市']!=area02]='其他'
        df_sp02['行政區'][df_sp02['縣市']!=area02]='其他'
        df_sp02['村里'][df_sp02['縣市']!=area02]='其他'
        df_sp02=df_sp02.groupby(['縣市','行政區','村里'])['id'].sum().reset_index()
        df_sp02=df_sp02.sort_values(by='id',ascending=False).reset_index(drop=True)
        df_sp02=pd.concat([df_sp02.loc[(df_sp02['村里']!='其他'),:],df_sp02.loc[(df_sp02['村里']=='其他'),:]]).reset_index(drop=True)
        df_sp02['rate']=df_sp02['id']/df_sp02['id'].sum()
        df_sp02.to_excel(writer02, sheet_name='顧客來源村里占比',index=0)

        df_sp01=pd.merge(df02,result)
        df_sp02=df_sp01.groupby(['縣市','行政區','村里','type'])['id'].count().reset_index()

        loss_rate_01=df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type1:new')].sum()/df_sp02['id'][(df_sp02['type']=='type1:new')].sum()
        loss_rate_02=df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type2:active')].sum()/df_sp02['id'][(df_sp02['type']=='type2:active')].sum()
        loss_rate_03=df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type3:1yr~3yr')].sum()/df_sp02['id'][(df_sp02['type']=='type3:1yr~3yr')].sum()
        loss_rate_04=df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type4:3yr~')].sum()/df_sp02['id'][(df_sp02['type']=='type4:3yr~')].sum()

        df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type1:new')]=df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type1:new')]/loss_rate_01
        df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type2:active')]=df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type2:active')]/loss_rate_02
        df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type3:1yr~3yr')]=df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type3:1yr~3yr')]/loss_rate_03
        df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type4:3yr~')]=df_sp02['id'][(df_sp02['行政區']!='NA')&(df_sp02['type']=='type4:3yr~')]/loss_rate_04

        df_sp02=df_sp02[df_sp02['行政區']!='NA'].reset_index(drop=True)
        df_sp02['縣市'][df_sp02['縣市']!=area02]='其他'
        df_sp02['行政區'][df_sp02['縣市']!=area02]='其他'
        df_sp02['村里'][df_sp02['縣市']!=area02]='其他'
        df_sp02=df_sp02.groupby(['縣市','行政區','type'])['id'].sum().reset_index()
        def replace_place(x):
            religion=['中正區','仁愛區','信義區','中山區','安樂區']
            if x =='NA':
                x=x
                return x
            elif x not in religion:
                x='其他'
                return x
            else:
                return x

        df_sp02['行政區']=df_sp02['行政區'].apply(replace_place)
        df_sp02=df_sp02.groupby(['行政區','type'])['id'].sum().reset_index()
        df_sp02=pd.concat([df_sp02.loc[(df_sp02['行政區']!='其他'),:],df_sp02.loc[(df_sp02['行政區']=='其他'),:]]).reset_index(drop=True)
        df_sp02['rate']=df_sp02['id']/df_sp02['id'].sum()
        df_sp02.to_excel(writer02, sheet_name='顧客來源X活躍程度',index=0)
        #writer02.save()
        #print(site[a]+'完成')
    
    ########################
    ########################
    if site[a]=='1G45基隆義一店':
        df_sp01=pd.merge(df02,result)
        df_sp01['行政區'][(df_sp01['縣市']!='基隆市')&(df_sp01['縣市']!='NA')]='其他'
        def replace_place(x):
            religion=['中正區','仁愛區','信義區','中山區','安樂區']
            if x =='NA':
                x=x
                return x
            elif x not in religion:
                x='其他'
                return x
            else:
                return x

        df_sp01['行政區']=df_sp01['行政區'].apply(replace_place)
        df_sp02=df_sp01.groupby(['行政區','type'])['id'].count().reset_index()
        df_sp02['rate']=df_sp02['id']/df_sp02['id'].sum()
        df_sp02.to_excel(writer02, sheet_name='顧客來源X活躍程度(NA獨立顯示)',index=0)
        writer02.save()
        print(site[a]+'完成')
    
    ########################
    
    
    ########################
    '''
    if site[a]=='1G45基隆義一店':
        df_sp01=pd.merge(df02,result)
        df_sp01['村里2']=df_sp01['村里']
        def replace_place02(x):
            religion=list(gdf.loc[(gdf['縣市']=='基隆市'),'村里'].values)
            if x =='NA':
                x=x
                return x
            elif x not in religion:
                x='其他'
                return x
            else:
                return x

        df_sp01['村里2']=df_sp01['村里2'].apply(replace_place02)
        df_sp02=df_sp01.groupby(['縣市','行政區','村里2','type'])['id'].count().reset_index()
        df_sp02['行政區'][df_sp02['村里2']=='其他']='其他'
        df_sp02['縣市'][df_sp02['村里2']=='其他']='其他'
        df_sp02['行政區'][df_sp02['村里2']=='NA']='NA'
        df_sp02['縣市'][df_sp02['村里2']=='NA']='NA'
        df_sp02=df_sp02.groupby(['縣市','行政區','村里2','type'])['id'].sum().reset_index()
        print(df_sp02.id.sum())

        df_sp02_loss=df_sp02['id'][df_sp02['村里2']!='NA'].sum()/df_sp02['id'].sum()
        df_sp02['id']=df_sp02['id']/df_sp02_loss
        df_sp02=df_sp02[df_sp02['村里2']!='NA'].reset_index(drop=True)
        df_sp02.to_excel('C:/Users/RT009223/TestriteGroup/滲透率專案/06_TLW社區店/基隆義一店客源分析.xlsx',index=False)
    '''
    ########################
    
    ########################
    '''
    if site[a]=='1G40三民澄清店':
        df_sp01=pd.merge(df02,result)
        df_sp01['行政區2']=df_sp01['行政區']
        def replace_place(x):
            religion=['三民區','鳳山區','苓雅區','鳥松區']
            if x =='NA':
                x=x
                return x
            elif x not in religion:
                x='其他'
                return x
            else:
                return x

        df_sp01['行政區2']=df_sp01['行政區2'].apply(replace_place)
        df_sp02=df_sp01.groupby(['type','行政區2'])['id'].count().reset_index()
        df_sp02.to_excel('C:/Users/RT009223/TestriteGroup/滲透率專案/06_TLW社區店/三民澄清店來客區域.xlsx',index=False)
    '''
    ########################
    
    result['村里']=result['村里'].apply(lambda x:x.replace('舘','館'))
    result['村里']=result['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
    result['村里']=result['村里'].apply(lambda x:x.replace('双','雙'))
    result['村里']=result['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
    result['村里']=result['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
    result['村里']=result['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))
    result['村里']=result['村里'].apply(lambda x:x.replace('欍','瓊'))

    result2=result.groupby(['縣市','行政區','村里'])['id'].count().reset_index()
    result2.columns=['縣市','行政區','村里','會員數']
    print(result2.shape)
    result2=pd.merge(pop,result2,how='outer')
    result2=result2[-(-result2['村里'].isin(pop['村里'])&-(result2['村里']=='NA'))].reset_index(drop=True)
    print(result2.shape)

    gdf_final=pd.merge(gdf,result2,how='right')
    gdf_final['會員數'][gdf_final['會員數'].isna()]=0
    gdf_final['site']=gdf_final['行政區']+'\n'+gdf_final['村里']
    gdf_final['ratio']=gdf_final['會員數']/gdf_final['戶口']
    print(gdf_final.shape)
    gdf_final.head()
    location=area[a]
    gdf_final_sub=gdf_final.loc[gdf_final['縣市'].isin(location),['縣市','行政區','村里','會員數','戶口','ratio']]
    print(gdf_final_sub.shape)

    df_final06_another=gdf_final_sub.groupby(['縣市','行政區'])['會員數','戶口'].sum().reset_index()
    df_final06_another['ratio']=df_final06_another['會員數']/df_final06_another['戶口']
    NA_value=gdf_final_sub_out[gdf_final_sub_out['行政區']=='NA']['會員數'].sum()
    not_NA_value=gdf_final_sub_out[gdf_final_sub_out['行政區']!='NA']['會員數'].sum()
    df_final06_another=pd.concat([df_final06_another,pd.DataFrame({'縣市':['其他地區','其他地區NA'],'會員數':[not_NA_value,NA_value]})])
    print(df_final06_another.shape)

    df_final07_another=gdf_final_sub.copy()
    NA_value=gdf_final_sub_out[gdf_final_sub_out['村里']=='NA']['會員數'].sum()
    not_NA_value=gdf_final_sub_out[gdf_final_sub_out['村里']!='NA']['會員數'].sum()
    df_final07_another=pd.concat([df_final07_another,pd.DataFrame({'縣市':['其他地區','其他地區NA'],'會員數':[not_NA_value,NA_value]})])
    print(df_final07_another.shape)

    df_final01_another['店點']=site[a]
    df_final01_another=df_final01_another.set_index('店點').reset_index()

    df_final02_another['店點']=site[a]
    df_final02_another=df_final02_another.set_index('店點').reset_index()

    df_final03_another['店點']=site[a]
    df_final03_another=df_final03_another.set_index('店點').reset_index()

    df_final04_another['店點']=site[a]
    df_final04_another=df_final04_another.set_index('店點').reset_index()

    df_final05_another['店點']=site[a]
    df_final05_another=df_final05_another.set_index('店點').reset_index()

    df_final06_another['店點']=site[a]
    df_final06_another=df_final06_another.set_index('店點').reset_index()

    df_final07_another['店點']=site[a]
    df_final07_another=df_final07_another.set_index('店點').reset_index()
    
    df_final05_2=df_final05_another.copy()
    df_final05_2['']=''
    df_final05_2=df_final05_2.set_index('').reset_index()
    
    df_final06_2=df_final06_another.copy()
    df_final06_2['']=''
    df_final06_2=df_final06_2.set_index('').reset_index()

    df_final07_2=df_final07_another.copy()
    df_final07_2['']=''
    df_final07_2=df_final07_2.set_index('').reset_index()

    df_final01=pd.concat([df_final01,df_final01_another])
    df_final02=pd.concat([df_final02,df_final02_another])
    df_final03=pd.concat([df_final03,df_final03_another])
    df_final04=pd.concat([df_final04,df_final04_another])
    
    df_final05a=pd.concat([df_final05a,df_final05_another])
    df_final06a=pd.concat([df_final06a,df_final06_another])
    df_final07a=pd.concat([df_final07a,df_final07_another])

    df_final05b=pd.concat([df_final05b, df_final05_2], axis=1)
    df_final06b=pd.concat([df_final06b, df_final06_2], axis=1)
    df_final07b=pd.concat([df_final07b, df_final07_2], axis=1)
    print('------------------------------------------------------')
    print('------------------------------------------------------')
    print('------------------------------------------------------')

1G32桃園大業店 202101
(2896, 7)
當月一般卡會員 (2109, 2)
(2109, 7)
1.0
1.0
1.0
1.0
(7203, 8)
(7203, 15)
(27, 3)
(22, 3)
一般卡會員(有完整地址) (9424, 4)
(9423, 7)
(7760, 4)
(9423, 7)
(0, 7)
(9423, 7)
(1, 7)
(1, 7)
(0, 7)
一般卡會員(有完整地址)_new (9424, 7)
一般卡會員(無完整地址) (2318, 3)
(436, 3)
(316, 3)
(133014, 4)
(316, 13)
(1, 13)
(271, 13)
(3, 13)
(41, 13)
(36, 13)
(5, 13)
(316, 4)
(316, 4)
(2002, 3)
(229, 5)
(1773, 5)
一般卡會員(無完整地址)_new (2318, 4)
(2318, 4)
全部一般卡會員_new (11742, 7)
1G32桃園大業店 have_share 0.8059104070856753
(1765, 4)
(7833, 6)
(7833, 9)
(1882, 6)
(60, 5)
(1884, 6)
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
1G33竹北文興店 202101
(3491, 7)
當月一般卡會員 (2702, 2)
(2702, 7)
1.0
0.9999999999999999
1.0
1.0
(11665, 8)
(11665, 15)
(28, 3)
(22, 3)
一般卡會員(有完整地址) (14869, 4)
(14865, 7)
(7760, 4)
(14862, 7)
(3, 7)
(3, 8)
(3, 12)
(14865, 7)
(4, 7)
(3, 7)
(1, 7)
(1, 8)
(1, 12)
(1, 7)
一般卡會員(有完整地址)_new (14869, 7)
一般卡會

(4077, 7)
(0, 7)
(4077, 7)
(0, 7)
(0, 7)
(0, 7)
一般卡會員(有完整地址)_new (4077, 7)
一般卡會員(無完整地址) (940, 3)
(198, 3)
(137, 3)
(133014, 4)
(137, 13)
(2, 13)
(112, 13)
(1, 13)
(22, 13)
(17, 13)
(5, 13)
(137, 4)
(137, 4)
(803, 3)
(129, 5)
(674, 5)
一般卡會員(無完整地址)_new (940, 4)
(940, 4)
全部一般卡會員_new (5017, 7)
1G42永和得和店 have_share 0.8162248355590991
(884, 4)
(7809, 6)
(7809, 9)
(2018, 6)
(56, 5)
(2020, 6)
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
1G46台中復興店 202101
(4347, 7)
當月一般卡會員 (3584, 2)
(3584, 7)
0.9999999999999999
0.9999999999999998
1.0
1.0
(13204, 8)
(13204, 15)
(25, 3)
(22, 3)
一般卡會員(有完整地址) (4649, 4)
(4649, 7)
(7760, 4)
(4649, 7)
(0, 7)
(4649, 7)
(0, 7)
(0, 7)
(0, 7)
一般卡會員(有完整地址)_new (4649, 7)
一般卡會員(無完整地址) (676, 3)
(244, 3)
(174, 3)
(133014, 4)
(174, 13)
(2, 13)
(148, 13)
(1, 13)
(23, 13)
(19, 13)
(4, 13)
(174, 4)
(174, 4)
(502, 3)
(104, 5)
(398, 5)
一般卡會員(無完整地址)_new (676, 4)
(676

In [302]:
result.drop_duplicates()

,id,addr,lon,lat,縣市,行政區,村里
0,20200609000604,桃園市龍潭區富林里21鄰民生路141巷57弄17號 ...,121.228534,24.846558,桃園市,龍潭區,富林里
1,20010316003228,桃園市平鎮區南勢里11鄰南華街85巷2號 ...,121.212012,24.915413,桃園市,平鎮區,南勢里
2,20020619000398,桃園市龍潭區建林里建國路15巷12號 325 ...,121.235113,24.841065,桃園市,龍潭區,建林里
3,20001217000658,桃園市龍潭區八德里31鄰梅龍路342巷6號 ...,121.195022,24.875608,桃園市,龍潭區,八德里
4,20010228004188,桃園市龍潭區龍星里31鄰神龍路201巷5號 ...,121.213644,24.860380,桃園市,龍潭區,龍星里
...,...,...,...,...,...,...,...
2112,20201003000574,NaN,NaN,NaN,NA,NA,NA
2113,20210123001047,NaN,NaN,NaN,NA,NA,NA
2114,20210124001534,NaN,NaN,NaN,NA,NA,NA
2115,20210126000194,NaN,NaN,NaN,NA,NA,NA


In [303]:
date[0]

'202101'

In [304]:
date[0][2:]

'2101'

In [ ]:
import os
dir='C:/Users/RT009223/TestriteGroup/滲透率專案/06_TLW社區店/month/'
Latest_Month=date[0][2:]
os.mkdir(os.path.join(site_dir+Latest_Month))

In [305]:
writer = pd.ExcelWriter('C:/Users/RT009223/TestriteGroup/滲透率專案/06_TLW社區店/month/'+date[0][2:]+'/社區店分析_分析用.xlsx', engine='xlsxwriter')

df_final01.to_excel(writer, sheet_name='臨時卡業績占比',index=0)
df_final02.to_excel(writer, sheet_name='會員組成',index=0)
df_final03.to_excel(writer, sheet_name='年齡',index=0)
df_final04.to_excel(writer, sheet_name='性別',index=0)
df_final05a.to_excel(writer, sheet_name='品類',index=0)
df_final06a.to_excel(writer, sheet_name='區滲透',index=0)
df_final07a.to_excel(writer, sheet_name='里滲透',index=0)

writer.save()

In [306]:
writer = pd.ExcelWriter('C:/Users/RT009223/TestriteGroup/滲透率專案/06_TLW社區店/month/'+date[0][2:]+'/社區店分析('+date[0]+')_Iris用.xlsx', engine='xlsxwriter')

df_final01.to_excel(writer, sheet_name='臨時卡業績占比',index=0)
df_final02.to_excel(writer, sheet_name='會員組成',index=0)
df_final03.to_excel(writer, sheet_name='年齡',index=0)
df_final04.to_excel(writer, sheet_name='性別',index=0)
df_final05b.to_excel(writer, sheet_name='品類',index=0)
df_final06b.to_excel(writer, sheet_name='區滲透',index=0)
df_final07b.to_excel(writer, sheet_name='里滲透',index=0)

writer.save()

In [307]:
'''
location=north1
gdf_final_sub=gdf_final.loc[gdf_final['縣市'].isin(location),['縣市','行政區','村里','會員數','戶口','ratio']]
gdf_final_sub.shape
'''

"\nlocation=north1\ngdf_final_sub=gdf_final.loc[gdf_final['縣市'].isin(location),['縣市','行政區','村里','會員數','戶口','ratio']]\ngdf_final_sub.shape\n"

In [308]:
#df_sp01=pd.merge(df02,result)

In [309]:
#df_sp01